In [ ]:
%run 02_retrieval.ipynb

In [ ]:
%run 03_rerankers.ipynb

In [ ]:
%run 04_router.ipynb

In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [12]:
generator_llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0,
    openai_api_key=os.getenv("MY_OPENAI_API_KEY")  
)

In [9]:
def build_context(reranked_docs, top_n=6):
    context = ""
    for doc in reranked_docs[:top_n]:
        context += doc.page_content + "\n\n"
    return context.strip()

In [8]:
GENERATION_PROMPT = """
You are an expert financial assistant. Answer the user's question STRICTLY based on the provided context from annual reports.

If the context does not contain the answer, say:
"I cannot find this information in the available documents."

--------------------
CONTEXT:
{context}
--------------------

QUESTION:
{query}

Final Answer:
"""


In [7]:
def generate_answer(query, reranked_docs):
    context = build_context(reranked_docs)

    prompt = GENERATION_PROMPT.format(
        context=context,
        query=query
    )

    response = generator_llm.invoke(prompt)
    return response.content


In [10]:
def answer_query(query):
    # Step 1: Route
    decision = route_query(query)
    print("Router decision:", decision)

    if decision == "direct":
        return generator_llm.invoke(query).content

    # Step 2: Retrieval
    retrieved = retrieve_candidates(query)

    # Step 3: Reranking
    reranked = rerank_documents(query, retrieved, top_k=6)

    # Step 4: Generation
    answer = generate_answer(query, reranked)
    return answer


In [13]:
query = "What did Microsoft report about cloud revenue in 2024?"

print(answer_query(query))


Router decision: rag
BM25 retrieved: 10 chunks
Semantic retrieved: 10 chunks
Combined unique: 17
Microsoft reported that its Microsoft Cloud revenue increased 23% to $137.4 billion in fiscal year 2024.
